In [1]:
import itertools

import networkx as nx
import numpy as np; np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import scipy.integrate as integrate

from simulate import ODE_simulate, GILL_simulate
from plot_module import plot_ODE, plot_gillespie, plot_network
from sim_param_from_network import names_from_network, gillespie_reaction_dict_from_network, dataframes_from_network, ODE_from_network
from network_generate import linear_network

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
MAX_T = 100000
TIME_POINTS = np.linspace(0, MAX_T, 1001)

In [16]:
@interact
def es(
    C_B = (0.00, 5, 0.01),
    BR_AXON = (0.00, 2, 0.005),
    DR = (0.025, 2.5, 0.01),
    GAMMA_ANT = (0, 100, 1),
    GAMMA_RET = (0, 100, 1),
    NSS = (50, 500, 50),
    ):

    bio_param = {'c_b':C_B, 'mu_a':BR_AXON, 'nss':NSS, 'delta':0, 'mu':DR, 'gamma_ant':GAMMA_ANT, 'gamma_ret':GAMMA_RET}
    G, VARS, COMP, START_STATE = linear_network(6, bio_param, start_pop = [100, 0])
    n_vars = len(VARS)
    
    ODE_model = ODE_from_network(G, prnt = False)
    ode_results = ODE_simulate(ODE_model, TIME_POINTS, START_STATE)
    print(np.round(ode_results[np.arange(0, n_vars, 2),-1],2))

interactive(children=(FloatSlider(value=2.5, description='C_B', max=5.0, step=0.01), FloatSlider(value=1.0, de…

In [4]:
ideal = np.array([250, 75, 75, 75, 75.0])

In [5]:
def end_state(time_points, c_b, br_axon, dr, gamma_ant, gamma_ret):
    
    bio_param = {'c_b':c_b, 'mu_a':br_axon, 'nss':250, 'delta':0, 'mu':dr, 'gamma_ant':gamma_ant, 'gamma_ret':gamma_ret}
    G, VARS, COMP, START_STATE = linear_network(5, bio_param, start_pop = [100, 0])
    n_vars = len(VARS)
    
    ODE_model = ODE_from_network(G, prnt = False)
    ode_results = ODE_simulate(ODE_model, time_points, START_STATE)
    
    return np.round(ode_results[np.arange(0, n_vars, 2),-1],2)

In [47]:
def new_bounds(cmin, cmax, optimal):
    newrange = (cmax-cmin)/3
    newmin = optimal-newrange if optimal-newrange > cmin else cmin
    newmax = optimal+newrange if optimal+newrange < cmax else cmax
    newmin = np.round(newmin, 2)
    newmax = np.round(newmax, 2)
    return newmin, newmax

In [35]:
def optimize(par_comb):
    lowest_dist = 100000000000000000
    lowest_dist_param = []

    for i, param in enumerate(par_comb):
        param = np.round(param, 2)
        es = end_state(TIME_POINTS, param[0], param[1], param[2], param[3], param[4])
        dist = euclidean_distances(es.reshape(1,-1), ideal.reshape(1,-1))

        print(i, end = '\r')

        if dist < lowest_dist:
            lowest_dist = dist
            lowest_dist_param = param
            print('          current best param:', lowest_dist_param, 'final state:', es, '                 ',end = '\r')
        
    return lowest_dist_param

In [50]:
min_cb = 0.01; max_cb = 5
min_br_axon = 0.01; max_br_axon = 1
min_dr = 0.025; max_dr = 2.5
min_gamma_ant = 1; max_gamma_ant = 40
min_gamma_ret = 1; max_gamma_ret = 40

steps = 4

for i in range(5):
    print("\nOptimization iteration",i+1)
    # generate parameter ranges to search
    range_cb = np.linspace(min_cb, max_cb, num=steps).tolist()
    range_br_axon = np.linspace(min_br_axon, max_br_axon, num=steps-2).tolist()
    range_dr = np.linspace(min_dr, max_dr, num=steps).tolist()
    range_gamma_ant = np.linspace(min_gamma_ant, max_gamma_ant, num=steps+2).tolist()
    range_gamma_ret = np.linspace(min_gamma_ret, max_gamma_ret, num=steps+2).tolist()

    # generate joint parameter space
    param_ranges = [range_cb, range_br_axon, range_dr, range_gamma_ant, range_gamma_ret]
    par_comb = list(itertools.product(*param_ranges))
    
    optimal_param = optimize(par_comb)
    
    min_cb, max_cb = new_bounds(min_cb, max_cb, optimal_param[0])
    min_br_axon, max_br_axon = new_bounds(min_br_axon, max_br_axon, optimal_param[1])
    min_dr, max_dr = new_bounds(min_dr, max_dr, optimal_param[2])
    min_gamma_ant, max_gamma_ant = new_bounds(min_gamma_ant, max_gamma_ant, optimal_param[3])
    min_gamma_ret, max_gamma_ret = new_bounds(min_gamma_ret, max_gamma_ret, optimal_param[4])
    
    print('\nnew parameter bounds:')
    print('cb:       \t', min_cb, '\t', max_cb)
    print('br_axon:  \t', min_br_axon, '\t',max_br_axon)
    print('dr:       \t', min_dr, '\t', max_dr)
    print('gamma_ant:\t', min_gamma_ant, '\t',max_gamma_ant) 
    print('gamma_ret:\t', min_gamma_ret, '\t', max_gamma_ret)
    
print('\nMost optimal parameters')
print(optimal_param)


Optimization iteration 1
1151      current best param: [1.67 0.01 2.5  8.8  8.8 ] final state: [246.4  147.84  91.27  60.63  47.22]                       
new parameter bounds:
cb:       	 0.01 	 3.33
br_axon:  	 0.01 	 0.34
dr:       	 1.67 	 2.5
gamma_ant:	 1 	 21.8
gamma_ret:	 1 	 21.8

Optimization iteration 2
1151      current best param: [1.12 0.01 1.95 5.16 5.16] final state: [246.2  135.74  76.57  46.35  33.64]                       
new parameter bounds:
cb:       	 0.01 	 2.23
br_axon:  	 0.01 	 0.12
dr:       	 1.67 	 2.23
gamma_ant:	 1 	 12.09
gamma_ret:	 1 	 12.09

Optimization iteration 3
1151      current best param: [0.75 0.01 2.04 3.22 1.  ] final state: [243.75 137.96  78.75  48.76  51.65]                  
new parameter bounds:
cb:       	 0.01 	 1.49
br_axon:  	 0.01 	 0.05
dr:       	 1.85 	 2.23
gamma_ant:	 1 	 6.92
gamma_ret:	 1 	 4.7

Optimization iteration 4
1151      current best param: [0.5  0.01 2.1  3.37 1.  ] final state: [240.22 137.37  79.22  49.59  53.